In [1]:
import re
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
node = []
allDatas = []
siganlList = []
SignalsName = []
messageName = []

In [2]:
def readFile(file_name):
    global node,allDatas,siganlList,SignalsName,messageName
    ''' 得到dbc文件的绝对路径'''
    filePath = file_name
    if filePath:
        print(filePath)
        f = open(filePath, "r")  # 设置文件对象
    else:
        print("读取文件失败！")
        return 0
    """
    NodesPattern:节点
    MessagePattern：消息
    SignalPattern：信号
    """
    NodesPattern = re.compile(r"BU_: (.*)", re.S)
    MessagePattern = re.compile(r"BO_ (.*?) (.*?): (.*?) (.*)", re.S)
#     SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@.*? \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@([0-9])([+|-]) \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    DefaultValue = '''BA_ "GenSig(.*?)" SG_ (\d+) signalname (\d+);'''

    line = f.readline()
    allDatas=[]
    while line:
        """ 匹配出节点 """
        NodesSearched = re.search(NodesPattern, line.strip())
        if NodesSearched:
            node = NodesSearched.group(1).split(" ")
            #print(node)
        """ 匹配出消息 """
        MessageSearched = re.search(MessagePattern, line.strip())
        if MessageSearched:
            siganlList.clear()
            """如果匹配到了message，则获取到message的相关参数 
             比如匹配到了NM_Message_ESC_409，则会解析出改message的一些参数构成list对象['1033', 'NM_Message_ESC_409', '8', 'ESC']
             这四个参数分别是 messgage ID ;message name ; messgae dataLenth ,message sender
             而且把这个list对象 加在了 siganlList 索引0的位置
            """
            Message = list(MessageSearched.groups())
            siganlList.append(Message)
            """ 只 要 message的名字 messageName 列表中"""
            messageName.append(Message[1])
            """读取下一行"""
            line = f.readline()
            """因为有些message并没有定义signal，所以 下一行还是message"""
            MessageSearched = re.search(MessagePattern, line.strip())
            SignalSearched = re.search(SignalPattern, line.strip())
            """下一行如果不是message的内容 就一定是signal的内容了"""
            if not MessageSearched:
                while SignalSearched:
                    """获取信号的参数追加到siganlList"""
                    signal = list(SignalSearched.groups())
                    siganlList.append(signal)
                    """只获取 signal name"""
                    SignalsName.append(signal[0])

                    # 再次解析信号，直到这个message下的信号全部解析完毕
                    line = f.readline()
                    SignalSearched = re.search(SignalPattern, line.strip())
           # print(siganlList)
            c = copy.deepcopy(siganlList)
            allDatas.append(c)
        else:
            line = f.readline()
            MessageSearched = re.search(MessagePattern, line.strip())
    f.close()  # 将文件关闭
    return 1

In [3]:
import os
file_names = []
path = os.path.abspath('../My_DBC')
# path = os.path.abspath('../opendbc-master/opendbc-master-delFalseData')
# path = os.path.abspath('./opendbc-master/opendbc-master')
dirs = os.listdir(path)                    # 获取指定路径下的文件
for i in dirs:
    if os.path.splitext(i)[1] == ".dbc":
        file_names.append(os.path.join(path,i))
file_names

['D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\My_DBC\\cadillac_ct6_object.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\My_DBC\\cadillac_ct6_powertrain.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\My_DBC\\gm_global_a_powertrain.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\My_DBC\\honda_civic_hatchback_ex_2017_can_generated.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\My_DBC\\honda_civic_sedan_16_diesel_2019_can_generated.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\My_DBC\\honda_civic_touring_2016_can_generated.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\My_DBC\\tesla_can.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\My_DBC\\tesla_powertrain.dbc',
 'D:\\--storage-

In [4]:
def get_id_labels(index):
    file_name = file_names[index]
    readFile(file_name)

    # 将DBC文件中的ID(10进制数)改为ID(16进制数)
    data_gts = {}
    for alldata in allDatas:
        data_gts[str(hex(int(alldata[0][0])))[2:]] = alldata

    # 得到DBC文件中各ID Message对应信号位置
    all_id_lsbs = {}
    for k in data_gts.keys():
        message = data_gts[k]
        lsb_list = []
        series = np.zeros(64)
        message_id = message[0][0]+"_"+message[0][1]
        del message[0]
        for signal in message:
            start = int(signal[1])
            length = int(signal[2])
            endianness = int(signal[3])
            start_row = int(start / 8)
            start_col = 8 - (start - start_row * 8) - 1
            new_start = start_row * 8 + start_col
            if endianness == 0:
                new_end = new_start + length
                cur_lsb = new_end - 1
            if endianness == 1:
                cur_lsb = new_start
            lsb_list.append(cur_lsb)
        all_id_lsbs[k] = lsb_list

    # 将各ID Message的信号位置改为 0/1 label
    all_id_labels = {}
    for k in all_id_lsbs:
        cur_lsb_list = all_id_lsbs[k]
        cur_label = np.zeros(64)
        for cur_lsb in cur_lsb_list:
            cur_label[cur_lsb] = 1
        all_id_labels[k] = cur_label
    return all_id_labels

In [5]:
def get_trace(data):
    group = data.groupby(by="ID")
    id_tracedict = {}
    for trace in list(group):
        id_tracedict[trace[0]] = trace[1]["bin"].apply(lambda x: list(x)).apply(lambda x: list(map(int, x))).tolist()
    return id_tracedict

In [6]:
def get_ids(id_tracedict_int, all_id_labels):
    ids = []
    for id in id_tracedict_int.keys():
        if(id_tracedict_int.__contains__(id) and all_id_labels.__contains__(id)):
            ids.append(id)
    return ids

In [7]:
import random
# 设置全局的随机种子
random.seed(100)
# 以id为单位shuffle
def get_train_test_ids(id_tracedict, all_id_labels):
    ids = get_ids(id_tracedict, all_id_labels)
    random.shuffle(ids)
    train_len = int(len(ids)*0.9)
    train_ids = ids[:train_len]
    test_ids = ids[train_len:]
    return train_ids, test_ids

In [8]:
def get_trace_data(id_tracedict_int, all_id_labels): 
    cur_trace_data = []
    cur_trace_label = []
    for id in id_tracedict_int.keys():
        cur_trace = id_tracedict_int[id]
        cur_label = all_id_labels[id]
        length = len(cur_trace)
        start = 4000
        end = start + 1000
        n = int((length-4000)/1000)
        for i in range(n):
            cur_trace_data.append(cur_trace[start: end])
            cur_trace_label.append(cur_label)
            start = end
            end = start + 1000
    return cur_trace_data, cur_trace_label

In [9]:
def get_data_label(id_tracedict_int, all_id_labels):
    cur_trace_data, cur_trace_label = get_trace_data(id_tracedict_int, all_id_labels)
    trace_indexs = pd.DataFrame(cur_trace_label).index.tolist()
    random.shuffle(trace_indexs)
    trace_data, trace_labels = np.array(cur_trace_data)[trace_indexs], np.array(cur_trace_label)[trace_indexs]
    return trace_data, trace_labels

In [10]:
def get_trace_data_dict(cur_ids, id_tracedict_int, all_id_labels, trace_data_dict, trace_label_dict): 
    for id in cur_ids:
        trace_data_dict[id] = id_tracedict_int[id]
        trace_label_dict[id] = all_id_labels[id]

In [11]:
# honda_dbcs = []
# cadillac_dbcs = []
# tesla_dbcs = []
# gm_dbcs = []
# for i in range(len(file_names)):
#     cur_name = file_names[i].split("\\")[-1].split("_")[0]
#     if cur_name == "honda":
#         honda_dbcs.append(i)
#     if cur_name == "cadillac":
#         cadillac_dbcs.append(i)
#     if cur_name == "tesla":
#         tesla_dbcs.append(i)
#     if cur_name == "gm":
#         gm_dbcs.append(i)

In [12]:
# honda_id_labels = {}
# for cur_dbc_index in honda_dbcs:
#     honda_id_labels.update(get_id_labels(cur_dbc_index)) 
# print(len(honda_id_labels)) 

In [13]:
# cadillac_id_labels = {}
# for cur_dbc_index in cadillac_dbcs:
#     cadillac_id_labels.update(get_id_labels(cur_dbc_index)) 
# for cur_dbc_index in gm_dbcs:
#     cadillac_id_labels.update(get_id_labels(cur_dbc_index)) 
# print(len(cadillac_id_labels)) 

In [12]:
# 合并所有Cadillac以及Chevy车的Ground Truth的 0/1 label
dict0 = get_id_labels(0)
dict1 = get_id_labels(1)
dict2 = get_id_labels(2)
cadillac_id_labels = dict()
for key in dict0.keys() | dict1.keys() | dict2.keys():
    for d in (dict0, dict1, dict2):
        if d.__contains__(key):
            cadillac_id_labels[key] = d[key]
            break

D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\My_DBC\cadillac_ct6_object.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\My_DBC\cadillac_ct6_powertrain.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\My_DBC\gm_global_a_powertrain.dbc


In [13]:
# 合并所有Honda车的Ground Truth的 0/1 label
dict3 = get_id_labels(3)
dict4 = get_id_labels(4)
dict5 = get_id_labels(5)
honda_id_labels = dict()
for key in dict3.keys() | dict4.keys() | dict5.keys():
    for d in (dict3, dict4, dict5):
        if d.__contains__(key):
            honda_id_labels[key] = d[key]
            break

D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\My_DBC\honda_civic_hatchback_ex_2017_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\My_DBC\honda_civic_sedan_16_diesel_2019_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\My_DBC\honda_civic_touring_2016_can_generated.dbc


In [14]:
# 得到Honda车的子数据集数据
data_hd_1 = pd.read_csv('../Data/Honda/honda001.csv')
data_hd_2 = pd.read_csv('../Data/Honda/honda002.csv')
# data_hd = pd.concat([data_hd_1,data_hd_2])
data_hd_3 = pd.read_csv('../Data/Honda/honda003.csv')
data_hd = pd.concat([data_hd_1,data_hd_2,data_hd_3])
# data_hd = pd.read_csv('../Data/Honda/honda001.csv')
data_hd["bin"] = data_hd["Data"].apply(int, base=16).apply(bin).str[2:].apply(lambda x: x.zfill(64))

# 得到Honda车子的数据集的大Trace dict
id_tracedict_hd = get_trace(data_hd)

In [15]:
# 得到用于训练和测试的id
train_ids_hd, test_ids_hd = get_train_test_ids(id_tracedict_hd, honda_id_labels)

In [16]:
# 得到Cadillac车的子数据集数据
data_cd_1 = pd.read_csv('../Data/Cadillac/CadillacCSV/001.txt.csv')
data_cd_2 = pd.read_csv('../Data/Cadillac/CadillacCSV/002.txt.csv')
data_cd_3 = pd.read_csv('../Data/Cadillac/CadillacCSV/003.txt.csv')
data_cd = pd.concat([data_cd_1,data_cd_2,data_cd_3])
# data_cd = pd.concat([data_cd_1,data_cd_2])
data_cd = pd.read_csv('../Data/Cadillac/CadillacCSV/001.txt.csv')
data_cd["bin"] = data_cd["Data"].apply(int, base=16).apply(bin).str[2:].apply(lambda x: x.zfill(64))

# 得到Honda车子的数据集的大Trace dict
id_tracedict_cd = get_trace(data_cd)

In [17]:
# 得到用于训练和测试的id
train_ids_cd, test_ids_cd = get_train_test_ids(id_tracedict_cd, cadillac_id_labels)

In [18]:
# 得到honda和caddilac车的trace dict及对应的label dict
train_data_dict = {}
train_label_dict = {}
get_trace_data_dict(train_ids_hd, id_tracedict_hd, honda_id_labels, train_data_dict, train_label_dict)
get_trace_data_dict(train_ids_cd, id_tracedict_cd, cadillac_id_labels, train_data_dict, train_label_dict)
# get_trace_data_dict(train_ids_ch, id_tracedict_ch, chevy_id_labels, train_data_dict, train_label_dict)

test_data_dict = {}
test_label_dict = {}
get_trace_data_dict(test_ids_hd, id_tracedict_hd, honda_id_labels, test_data_dict, test_label_dict)
get_trace_data_dict(test_ids_cd, id_tracedict_cd, cadillac_id_labels, test_data_dict, test_label_dict)
# get_trace_data_dict(test_ids_ch, id_tracedict_cd, cadillac_id_labels, test_data_dict, test_label_dict)

In [19]:
# shuffle后得到训练集和测试集，其中训练集的id占70%，测试集的id占30%
train_data, train_labels = get_data_label(train_data_dict, train_label_dict)

In [20]:
test_data, test_labels = get_data_label(test_data_dict, test_label_dict)

In [34]:
# train_data = np.concatenate((train_data_hd, train_data_cd), axis=0)
# train_labels = np.concatenate((train_labels_hd, train_labels_cd), axis=0)

# test_data = np.concatenate((test_data_hd, test_data_cd), axis=0)
# test_labels = np.concatenate((test_labels_hd, test_labels_cd), axis=0)

In [23]:
# data_labels_0 = np.zeros(64)
# data_labels_1 = np.zeros(64)
# for i in range(len(train_labels)):
#     cur_label = train_labels[i]
#     for j in range(64):
#         temp = cur_label[j]
#         if temp == 0:
#             data_labels_0[j] = data_labels_0[j]+1
#         else:
#             data_labels_1[j] = data_labels_1[j]+1        

In [24]:
# data_labels_0

In [ ]:
# data_labels_1

In [ ]:
# import matplotlib.pyplot as plt
# # fig = plt.fig(figsize(10,5))
# plt.plot(data_labels_0)
# plt.plot(data_labels_1)

In [ ]:
# import matplotlib.pyplot as plt
# fig = plt.figure(figsize=(13,5))
# xis = np.arange(0,64)
# plt.bar(xis, data_labels_0)
# plt.bar(xis, data_labels_1)

In [ ]:
# data_labels_0 = np.zeros(64)
# data_labels_1 = np.zeros(64)
# for i in range(len(test_labels)):
#     cur_label = test_labels[i]
#     for j in range(64):
#         temp = cur_label[j]
#         if temp == 0:
#             data_labels_0[j] = data_labels_0[j]+1
#         else:
#             data_labels_1[j] = data_labels_1[j]+1   
# fig = plt.figure(figsize=(13,5))
# xis = np.arange(0,64)
# plt.bar(xis, data_labels_0)
# plt.bar(xis, data_labels_1)

In [ ]:
# data_labels_0

In [ ]:
# data_labels_1

In [21]:
import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, Input, Permute, BatchNormalization, Flatten, MaxPooling1D, Conv1D,Conv2D
from keras.layers import concatenate, add
from keras.losses import mean_absolute_percentage_error, mean_absolute_error
from sklearn.metrics import mean_squared_error
import keras.backend as K
from sklearn import metrics

Using TensorFlow backend.
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a f

### 1.改进模型一：修改conv

In [22]:
inputs = Input(shape = (1000,64,1))
x = keras.layers.Conv2D(15, 3, padding='same', data_format='channels_last')(inputs)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(64, activation='sigmoid')(x)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 64, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1000, 64, 15)      150       
_________________________________________________________________
flatten_1 (Flatten)          (None, 960000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                61440064  
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
Total params: 61,444,374
Trainable params: 61,444,374
Non-trainable params: 0
_________________________________________________________________


In [23]:
# 从(None, 1000, 64)变成(None, 1000, 64, 1)
train_data_4 = np.expand_dims(train_data, 3)

In [24]:
model.fit(train_data_4, train_labels,epochs=5, batch_size=10)


Epoch 1/5


ResourceExhaustedError: OOM when allocating tensor with shape[960000,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node training/Adam/Square_2}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
test_data_4 = np.expand_dims(test_data, 3)

In [43]:
print("训练集:")
predict_target_msb_prob = model.predict(train_data_4)
predict_target_msb_label = (predict_target_msb_prob > 0.5).astype(int)
predict_target_msb_1D = predict_target_msb_label.flatten()
train_labels_1D = train_labels.flatten()
print("预测正确数量,训练集样本量:")
print(sum(predict_target_msb_1D == train_labels_1D),len(train_labels_1D))
print("训练集精确度等指标：")
print(metrics.classification_report(train_labels_1D,predict_target_msb_1D))
print("混淆矩阵：")
print(metrics.confusion_matrix(train_labels_1D,predict_target_msb_1D))

print("测试集:")
predict_target_test_msb_prob =model.predict(test_data_4)
predict_target_test_msb_label = (predict_target_test_msb_prob > 0.5).astype(int)
predict_target_test_msb_1D = predict_target_test_msb_label.flatten()
test_labels_1D = test_labels.flatten()
print("预测正确数量,训练集样本量:")
print(sum(predict_target_test_msb_1D == test_labels_1D),len(test_labels_1D))
print("测试集精确度等指标：")
print(metrics.classification_report(test_labels_1D,predict_target_test_msb_1D))
print("混淆矩阵：")
print(metrics.confusion_matrix(test_labels_1D,predict_target_test_msb_1D))

训练集:
预测正确数量,训练集样本量:
53481 54784
训练集精确度等指标：
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     50900
         1.0       0.96      0.69      0.80      3884

    accuracy                           0.98     54784
   macro avg       0.97      0.84      0.90     54784
weighted avg       0.98      0.98      0.97     54784

混淆矩阵：
[[50799   101]
 [ 1202  2682]]
测试集:
预测正确数量,训练集样本量:
6881 7424
测试集精确度等指标：
              precision    recall  f1-score   support

         0.0       0.95      0.97      0.96      6935
         1.0       0.43      0.32      0.37       489

    accuracy                           0.93      7424
   macro avg       0.69      0.65      0.66      7424
weighted avg       0.92      0.93      0.92      7424

混淆矩阵：
[[6724  211]
 [ 332  157]]


### 2.改进模型二：修改conv + add bit-flip

In [35]:
# 使用READ方法计算bit-flip
def get_bit_Flips(cur_id_traces):
    bit_Flips = []
    for i in range(len(cur_id_traces)):
        messagelist = cur_id_traces[i]
        trace_len = len(messagelist)
        bit_flip = np.zeros(64)
        previous = messagelist[0]
        for item in messagelist:
            for ix in range(64):
                if item[ix] != previous[ix]:
                    bit_flip[ix] = bit_flip[ix] + 1
            previous = item
        for ix in range(64):
            bit_flip[ix] = bit_flip[ix] / trace_len
        bit_Flips.append(bit_flip)
    return bit_Flips

In [36]:
train_data_bitfilps = get_bit_Flips(train_data)
test_data_bitfilps = get_bit_Flips(test_data)

In [39]:
train_data_bitfilps = np.array(train_data_bitfilps)
test_data_bitfilps = np.array(test_data_bitfilps)

In [30]:
inputs1 = Input(shape = (1000,64,1))
inputs2 = Input(shape = (64,))
x = Conv2D(15, 3, padding='same', data_format='channels_last')(inputs1)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = add([x, inputs2])
outputs = Dense(64, activation='sigmoid')(x)
model_2 = Model(inputs=[inputs1,inputs2], outputs=outputs)
model_2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_2.summary()




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 1000, 64, 1)  0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 1000, 64, 15) 150         input_6[0][0]                    
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 960000)       0           conv2d_3[0][0]                   
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           61440064    flatten_1[0][0]              

In [40]:
model_2.fit([train_data_4, train_data_bitfilps], train_labels,epochs=5, batch_size=10)


Epoch 1/5
1875/1875 [==============================] - 163s 87ms/step - loss: 1.5653 - acc: 0.8963
Epoch 2/5
1875/1875 [==============================] - 167s 89ms/step - loss: 0.9060 - acc: 0.9412
Epoch 3/5
1875/1875 [==============================] - 155s 83ms/step - loss: 0.6993 - acc: 0.9555
Epoch 4/5
1875/1875 [==============================] - 159s 85ms/step - loss: 0.5983 - acc: 0.9619
Epoch 5/5
1875/1875 [==============================] - 150s 80ms/step - loss: 0.3862 - acc: 0.9754


In [45]:
print("训练集:")
predict_target_msb_prob = model_2.predict([train_data_4, train_data_bitfilps])
predict_target_msb_label = (predict_target_msb_prob > 0.5).astype(int)
predict_target_msb_1D = predict_target_msb_label.flatten()
train_labels_1D = train_labels.flatten()
print("预测正确数量,训练集样本量:")
print(sum(predict_target_msb_1D == train_labels_1D),len(train_labels_1D))
print("训练集精确度等指标：")
print(metrics.classification_report(train_labels_1D,predict_target_msb_1D))
print("混淆矩阵：")
print(metrics.confusion_matrix(train_labels_1D,predict_target_msb_1D))

print("测试集:")
predict_target_test_msb_prob =model_2.predict([test_data_4, test_data_bitfilps])
predict_target_test_msb_label = (predict_target_test_msb_prob > 0.5).astype(int)
predict_target_test_msb_1D = predict_target_test_msb_label.flatten()
test_labels_1D = test_labels.flatten()
print("预测正确数量,训练集样本量:")
print(sum(predict_target_test_msb_1D == test_labels_1D),len(test_labels_1D))
print("测试集精确度等指标：")
print(metrics.classification_report(test_labels_1D,predict_target_test_msb_1D))
print("混淆矩阵：")
print(metrics.confusion_matrix(test_labels_1D,predict_target_test_msb_1D))

训练集:
预测正确数量,训练集样本量:
117076 120000
训练集精确度等指标：
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99    111450
         1.0       0.98      0.67      0.80      8550

    accuracy                           0.98    120000
   macro avg       0.98      0.83      0.89    120000
weighted avg       0.98      0.98      0.97    120000

混淆矩阵：
[[111348    102]
 [  2822   5728]]
测试集:
预测正确数量,训练集样本量:
15036 16128
测试集精确度等指标：
              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96     15078
         1.0       0.48      0.49      0.48      1050

    accuracy                           0.93     16128
   macro avg       0.72      0.73      0.72     16128
weighted avg       0.93      0.93      0.93     16128

混淆矩阵：
[[14523   555]
 [  537   513]]


### 3.改进模型二：修改conv + loss

In [ ]:
inputs = Input(shape = (1000,64,1))
x = keras.layers.Conv2D(15, 3, padding='same', data_format='channels_last')(inputs)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(64, activation='sigmoid')(x)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

### 原模型

In [48]:
def CNN_Model(train_data, train_labels, test_data, test_labels):
    #构建CNN分类模型(LeNet-5)
    #模型结构：嵌入-卷积池化*2-dropout-BN-全连接-dropout-全连接
    model_msb = Sequential()
    model_msb.add(Conv1D(256, 5, padding='same', input_shape = (1000,64)))
    model_msb.add(MaxPooling1D(3, 3, padding='same'))
    model_msb.add(Conv1D(128, 5, padding='same'))
    model_msb.add(MaxPooling1D(3, 3, padding='same'))
    model_msb.add(Conv1D(64, 3, padding='same'))
    model_msb.add(Flatten())
    model_msb.add(Dropout(0.1))
    model_msb.add(BatchNormalization())  # (批)规范化层
    model_msb.add(Dense(256, activation='relu'))
    model_msb.add(Dropout(0.1))
    model_msb.add(Dense(64, activation='sigmoid'))
    # model_msb.compile(loss=my_loss,optimizer='adam',metrics=['accuracy'])
    model_msb.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    # one_hot_labels = keras.utils.to_categorical(y_train, num_classes=3)  # 将标签转换为one-hot编码
    model_msb.summary()
    
    model_msb.fit(train_data, train_labels,epochs=5, batch_size=10)
    
    print("训练集:")
    predict_target_msb_prob = model_msb.predict(train_data)
    predict_target_msb_label = (predict_target_msb_prob > 0.5).astype(int)
    predict_target_msb_1D = predict_target_msb_label.flatten()
    train_labels_1D = train_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_msb_1D == train_labels_1D),len(train_labels_1D))
    print("训练集精确度等指标：")
    print(metrics.classification_report(train_labels_1D,predict_target_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(train_labels_1D,predict_target_msb_1D))

    print("测试集:")
    predict_target_test_msb_prob =model_msb.predict(test_data)
    predict_target_test_msb_label = (predict_target_test_msb_prob > 0.5).astype(int)
    predict_target_test_msb_1D = predict_target_test_msb_label.flatten()
    test_labels_1D = test_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_test_msb_1D == test_labels_1D),len(test_labels_1D))
    print("测试集精确度等指标：")
    print(metrics.classification_report(test_labels_1D,predict_target_test_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(test_labels_1D,predict_target_test_msb_1D))
    return model_msb

In [49]:
model = CNN_Model(train_data, train_labels, test_data, test_labels)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 256)         82176     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 334, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 334, 128)          163968    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 112, 128)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 112, 64)           24640     
_________________________________________________________________
flatten_4 (Flatten)          (None, 7168)              0         
____

In [43]:
123840+12288

136128

In [154]:
bit_Flips = get_bit_Flips(test_data)

In [155]:
preds = model.predict(test_data)

In [80]:
# for i in range(len(bit_Flips)):
#     cur_bit_flip = bit_Flips[i]
#     new_labels = []
#     new_preds = []
#     for j in range(len(test_labels[i])):
#         next_j = j+1
#         if cur_trace_label[i][j] == 1:
#             if cur_bit_flip[j] != 0:
#                 if cur_bit_flip[next_j] != 0:
#                     new_labels.append(cur_trace_label[i][j])
#                     new_preds.append(preds[i][j])
#     new_labels = np.array(new_labels)
#     new_preds = np.array(new_preds)

In [130]:
def get_score0(test_labels, bit_Flips):
    new_labels = test_labels
    new_preds = preds
    for i in range(len(new_preds)):
        for j in range(64):
            if new_preds[i][j] >= 0.5:
                new_preds[i][j] = 1
            if new_preds[i][j] < 0.5:
                new_preds[i][j] = 0
    for i in range(len(test_labels)):
        for j in range(64):
            next_j = j+1
            last_j = j-1
            if test_labels[i][j] == 1:
                if bit_Flips[i][j] != 0:
                    if next_j < 64 and bit_Flips[i][next_j] != 0:
                        if last_j >= 0 and bit_Flips[i][last_j] != 0:
                            new_labels[i][j] = 2
                            if new_preds[i][j] == 1:
                                new_preds[i][j] = 2
    return new_labels.flatten(),new_preds.flatten()

In [156]:
def get_score1(test_labels, preds, bit_Flips):
    new_labels = []
    new_preds = []
    for i in range(len(test_labels)):
        for j in range(64):
            next_j = j+1
            if test_labels[i][j] == 1:
                if bit_Flips[i][j] != 0:
                    if next_j < 64 and bit_Flips[i][next_j] != 0:
                        new_labels.append(test_labels[i][j])
                        if preds[i][j] >= 0.5:
                            new_preds.append(1)
                        if preds[i][j] < 0.5:
                            new_preds.append(0)
                    if next_j > 64:
                        new_labels.append(test_labels[i][j])
                        if preds[i][j] >= 0.5:
                            new_preds.append(1)
                        if preds[i][j] < 0.5:
                            new_preds.append(0)
    new_labels = np.array(new_labels)
    new_preds = np.array(new_preds)
    return new_labels,new_preds

In [160]:
new_labels = []
new_preds = []
for i in range(len(test_labels)):
    for j in range(64):
        next_j = j+1
        if test_labels[i][j] == 1:
            if bit_Flips[i][j] != 0:
                if next_j < 64 and bit_Flips[i][next_j] != 0:
                    new_labels.append(test_labels[i][j])
#                     if preds[i][j] >= 0.5:
#                         new_preds.append(1)
#                     if preds[i][j] < 0.5:
#                         new_preds.append(0)
                if next_j > 64:
                    new_labels.append(test_labels[i][j])
#                     if preds[i][j] >= 0.5:
#                         new_preds.append(1)
#                     if preds[i][j] < 0.5:
#                         new_preds.append(0)
new_labels = np.array(new_labels)
new_preds = np.array(new_preds)

In [161]:
new_labels

array([], dtype=float64)

In [100]:
def get_score2(test_labels, bit_Flips):
    new_labels = []
    new_preds = []
    for i in range(len(test_labels)):
        for j in range(64):
            next_j = j+1
            if bit_Flips[i][j] != 0:
                if next_j < 64 and bit_Flips[i][next_j] != 0:
                    new_labels.append(test_labels[i][j])
                    if preds[i][j] >= 0.5:
                        new_preds.append(1)
                    if preds[i][j] < 0.5:
                        new_preds.append(0)
    new_labels = np.array(new_labels)
    new_preds = np.array(new_preds)
    return new_labels,new_preds

In [110]:
def get_score3(test_labels, bit_Flips):
    new_labels = []
    new_preds = []
    for i in range(len(test_labels)):
        for j in range(64):
            next_j = j+1
            if test_labels[i][j] == 1:
                new_labels.append(test_labels[i][j])
                if preds[i][j] >= 0.5:
                    new_preds.append(1)
                if preds[i][j] < 0.5:
                    new_preds.append(0)
    new_labels = np.array(new_labels)
    new_preds = np.array(new_preds)
    return new_labels,new_preds

In [132]:
import sklearn
test_new_labels,test_new_preds = get_score0(test_labels, bit_Flips)
print("预测正确数量,训练集样本量:")
print(sum(test_new_preds == test_new_labels),len(test_new_labels))
print("测试集精确度等指标：")
print(sklearn.metrics.classification_report(test_new_labels,test_new_preds))
print("混淆矩阵：")
print(sklearn.metrics.confusion_matrix(test_new_labels,test_new_preds))

预测正确数量,训练集样本量:
16294 17920
测试集精确度等指标：
              precision    recall  f1-score   support

         0.0       0.98      0.93      0.96     17254
         1.0       0.12      0.38      0.18       477
         2.0       0.00      0.00      0.00       189

    accuracy                           0.91     17920
   macro avg       0.37      0.44      0.38     17920
weighted avg       0.95      0.91      0.93     17920

混淆矩阵：
[[16112  1142     0]
 [  295   182     0]
 [   11   178     0]]


D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 

In [99]:
import sklearn
test_new_labels,test_new_preds = get_score1(test_labels, bit_Flips)
print("预测正确数量,训练集样本量:")
print(sum(test_new_preds == test_new_labels),len(test_new_labels))
print("测试集精确度等指标：")
print(sklearn.metrics.classification_report(test_new_labels,test_new_preds))
print("混淆矩阵：")
print(sklearn.metrics.confusion_matrix(test_new_labels,test_new_preds))

预测正确数量,训练集样本量:
178 189
测试集精确度等指标：
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.94      0.97       189

    accuracy                           0.94       189
   macro avg       0.50      0.47      0.49       189
weighted avg       1.00      0.94      0.97       189

混淆矩阵：
[[  0   0]
 [ 11 178]]


D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with 

In [103]:
import sklearn
test_new_labels,test_new_preds = get_score2(test_labels, bit_Flips)
print("预测正确数量,训练集样本量:")
print(sum(test_new_preds == test_new_labels),len(test_new_labels))
print("测试集精确度等指标：")
print(sklearn.metrics.classification_report(test_new_labels,test_new_preds))
print("混淆矩阵：")
print(sklearn.metrics.confusion_matrix(test_new_labels,test_new_preds))

预测正确数量,训练集样本量:
5576 5713
测试集精确度等指标：
              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99      5524
         1.0       0.59      0.94      0.72       189

    accuracy                           0.98      5713
   macro avg       0.79      0.96      0.85      5713
weighted avg       0.98      0.98      0.98      5713

混淆矩阵：
[[5398  126]
 [  11  178]]


In [112]:
import sklearn
test_new_labels,test_new_preds = get_score3(test_labels, bit_Flips)
print("预测正确数量,训练集样本量:")
print(sum(test_new_preds == test_new_labels),len(test_new_labels))
print("测试集精确度等指标：")
print(sklearn.metrics.classification_report(test_new_labels,test_new_preds))
print("混淆矩阵：")
print(sklearn.metrics.confusion_matrix(test_new_labels,test_new_preds))

预测正确数量,训练集样本量:
360 666
测试集精确度等指标：
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.54      0.70       666

    accuracy                           0.54       666
   macro avg       0.50      0.27      0.35       666
weighted avg       1.00      0.54      0.70       666

混淆矩阵：
[[  0   0]
 [306 360]]


D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\--storage--\program\application\windows-installer\Miniconda3\Miniconda3\envs\Commonl_Py3.6\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with 

In [114]:
test_new_preds

array([0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,

In [47]:
#构建TextCNN模型
#模型结构：词嵌入-卷积池化*3-拼接-全连接-dropout-全连接
def TextCNN_model_1(train_data, train_labels, test_data, test_labels):
    main_input = Input(shape=(1000,64))
    # 词窗大小分别为3,4,5
    cnn1 = Conv1D(128, 4, padding='same', strides=1, activation='relu')(main_input)
    cnn1 = MaxPooling1D(pool_size=64)(cnn1)
    cnn2 = Conv1D(128, 8, padding='same', strides=1, activation='relu')(main_input)
    cnn2 = MaxPooling1D(pool_size=64)(cnn2)
    cnn3 = Conv1D(128, 16, padding='same', strides=1, activation='relu')(main_input)
    cnn3 = MaxPooling1D(pool_size=64)(cnn3)
    # 合并三个模型的输出向量
    cnn = concatenate([cnn1, cnn2, cnn3], axis=-1)
    flat = Flatten()(cnn)
    drop = Dropout(0.2)(flat)
    main_output = Dense(64, activation='sigmoid')(drop)
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    K.set_value(model.optimizer.lr,1e-4)
    model.fit(train_data, train_labels,epochs=5, batch_size=10)
    
    print("训练集:")
    predict_target_msb_prob = model.predict(train_data)
    predict_target_msb_label = (predict_target_msb_prob > 0.5).astype(int)
    predict_target_msb_1D = predict_target_msb_label.flatten()
    train_labels_1D = train_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_msb_1D == train_labels_1D),len(train_labels_1D))
    print("训练集精确度等指标：")
    print(metrics.classification_report(train_labels_1D,predict_target_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(train_labels_1D,predict_target_msb_1D))

    print("测试集:")
    predict_target_test_msb_prob =model.predict(test_data)
    predict_target_test_msb_label = (predict_target_test_msb_prob > 0.5).astype(int)
    predict_target_test_msb_1D = predict_target_test_msb_label.flatten()
    test_labels_1D = test_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_test_msb_1D == test_labels_1D),len(test_labels_1D))
    print("测试集精确度等指标：")
    print(metrics.classification_report(test_labels_1D,predict_target_test_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(test_labels_1D,predict_target_test_msb_1D))

In [48]:
TextCNN_model_1(train_data, train_labels, test_data, test_labels)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000, 64)     0                                            
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 1000, 128)    32896       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_14 (Conv1D)              (None, 1000, 128)    65664       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 1000, 128)    131200      input_1[0][0]                    
____________________________________________________________________________________________

In [63]:
TextCNN_model_1(train_data, train_labels, test_data, test_labels)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000, 64)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1000, 128)    32896       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 1000, 128)    65664       input_1[0][0]                    
__________________________________________________________________________

In [46]:
# 120000/136128

In [53]:
def my_loss(y_true, y_pred):
#     temp = lambda(x )
    temp = y_pred
    right_count = 0
    for i in range(64):
        if(y_true[i] == 1):
            if(temp[i] == 1):
                right_count = right_count + 1
    return right_count/64

In [89]:
def multilabel_categorical_crossentropy(y_true, y_pred):
    """多标签分类的交叉熵
    说明：y_true和y_pred的shape一致，y_true的元素非0即1，
         1表示对应的类为目标类，0表示对应的类为非目标类。
    """
#     print("y_pred:")
#     print(y_pred.shape)
#     print("y_true:")
#     print(y_true.shape)
    y_pred = (1 - 2 * y_true) * y_pred
#     print("y_pred:")
#     print(y_pred.shape)
    y_pred_neg = y_pred - y_true * 1e12
    y_pred_pos = y_pred - (1 - y_true) * 1e12
    zeros = K.zeros_like(y_pred[..., :1])
    y_pred_neg = K.concatenate([y_pred_neg, zeros], axis=-1)
    y_pred_pos = K.concatenate([y_pred_pos, zeros], axis=-1)
    neg_loss = K.logsumexp(y_pred_neg, axis=-1)
    pos_loss = K.logsumexp(y_pred_pos, axis=-1)
    print(pos_loss.shape)
    return neg_loss + pos_loss

In [90]:
#构建TextCNN模型
#模型结构：词嵌入-卷积池化*3-拼接-全连接-dropout-全连接
def TextCNN_model_2(train_data, train_labels, test_data, test_labels):
    main_input = Input(shape=(1000,64))
    # 词窗大小分别为3,4,5
    cnn1 = Conv1D(128, 4, padding='same', strides=1, activation='relu')(main_input)
    cnn1 = MaxPooling1D(pool_size=64)(cnn1)
    cnn2 = Conv1D(128, 8, padding='same', strides=1, activation='relu')(main_input)
    cnn2 = MaxPooling1D(pool_size=64)(cnn2)
    cnn3 = Conv1D(128, 16, padding='same', strides=1, activation='relu')(main_input)
    cnn3 = MaxPooling1D(pool_size=64)(cnn3)
    # 合并三个模型的输出向量
    cnn = concatenate([cnn1, cnn2, cnn3], axis=-1)
    flat = Flatten()(cnn)
    drop = Dropout(0.2)(flat)
    main_output = Dense(64, activation='softmax')(drop)
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss=multilabel_categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
    model.summary()

    K.set_value(model.optimizer.lr,1e-4)
    model.fit(train_data, train_labels,epochs=1, batch_size=10)
    
    print("训练集:")
    predict_target_msb_prob = model.predict(train_data)
    predict_target_msb_label = (predict_target_msb_prob > 0.5).astype(int)
    predict_target_msb_1D = predict_target_msb_label.flatten()
    train_labels_1D = train_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_msb_1D == train_labels_1D),len(train_labels_1D))
    print("训练集精确度等指标：")
    print(metrics.classification_report(train_labels_1D,predict_target_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(train_labels_1D,predict_target_msb_1D))

    print("测试集:")
    predict_target_test_msb_prob =model.predict(test_data)
    predict_target_test_msb_label = (predict_target_test_msb_prob > 0.5).astype(int)
    predict_target_test_msb_1D = predict_target_test_msb_label.flatten()
    test_labels_1D = test_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_test_msb_1D == test_labels_1D),len(test_labels_1D))
    print("测试集精确度等指标：")
    print(metrics.classification_report(test_labels_1D,predict_target_test_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(test_labels_1D,predict_target_test_msb_1D))

In [91]:
TextCNN_model_2(train_data, train_labels, test_data, test_labels)

y_pred:
(?, 64)
y_true:
(?, ?)
y_pred:
(?, 64)
(?,)


In [55]:
import joblib
# TextCNN大端模型保存
joblib.dump(filename='./Model/TextCNN_MSB.model', value=model_msb)

['./Model/TextCNN_MSB.model']

In [40]:
# lsb边界是否识别正确的准确度统计

right_count = 0
total_count = 0
for i in range(len(test_labels_1D)):
    if(test_labels_1D[i] == 1):
        total_count = total_count + 1
        if(predict_target_test_msb_1D[i] == 1):
            right_count = right_count + 1
acc = right_count / total_count

In [41]:
acc

0.3140814081408141

In [48]:
def get_locate_label_array(test_label_array):
    test_label_locate_array = []
    for id in range(len(test_label_array)):
        temp_list = test_label_array[id]
        cur_label = []
        for i in range(64):
            if(temp_list[i] == 1):
                cur_label.append(i)
        test_label_locate_array.append(cur_label)
    return test_label_locate_array

In [49]:
final_label_array = get_locate_label_array(predict_target_test_msb_label)
test_label_locate_array = get_locate_label_array(test_labels)

In [50]:
for id in range(len(final_label_array)):
    print(final_label_array[id])
    print(test_label_locate_array[id])
    print("------------------------------------------")

[59, 63]
[23, 39]
------------------------------------------
[15, 31, 47, 55, 59, 63]
[15, 27, 41, 42, 59, 63]
------------------------------------------
[7, 47, 63]
[15, 23]
------------------------------------------
[59, 63]
[23, 39]
------------------------------------------
[39, 41, 47, 59, 63]
[15]
------------------------------------------
[39, 59, 63]
[7, 31, 32, 33, 38, 39, 49, 50, 55, 59, 63]
------------------------------------------
[]
[15]
------------------------------------------
[39, 63]
[15, 31, 59, 63]
------------------------------------------
[39, 59, 63]
[15, 31, 59, 63]
------------------------------------------
[7, 15, 31, 47, 59, 63]
[15, 23]
------------------------------------------
[39, 59, 63]
[15, 31, 59, 63]
------------------------------------------
[]
[15]
------------------------------------------
[7, 8, 15, 31, 47, 49, 55, 59, 63]
[55]
------------------------------------------
[15, 47, 55, 59, 63]
[23, 39]
------------------------------------------
[39